In [2]:
# Copyright (c) 2024 Graphcore Ltd. All rights reserved.

from pandas import DataFrame
import numpy as np

In [20]:
from gfloat import decode_float
from gfloat.formats import *

# Print all values in a float8 datatype
fmt = format_info_ocp_e5m2
vals = [decode_float(fmt, i) for i in range(256)]
DataFrame(vals).set_index("ival").drop(columns=["fi"])

,fval,valstr,exp,expval,significand,fsignificand,signbit,fclass
ival,,,,,,,,
0,0.000000e+00,0.0,0,-14,0,0.00,0,FloatClass.ZERO
1,1.525879e-05,~1.5258789e-05,0,-14,1,0.25,0,FloatClass.SUBNORMAL
2,3.051758e-05,~3.0517578e-05,0,-14,2,0.50,0,FloatClass.SUBNORMAL
3,4.577637e-05,~4.5776367e-05,0,-14,3,0.75,0,FloatClass.SUBNORMAL
4,6.103516e-05,~6.1035156e-05,1,-14,0,1.00,0,FloatClass.NORMAL
...,...,...,...,...,...,...,...,...
251,-5.734400e+04,-57344.0,30,15,3,1.75,1,FloatClass.NORMAL
252,-inf,-inf,31,16,0,1.00,1,FloatClass.INFINITE
253,NaN,nan,31,16,1,1.25,1,FloatClass.NAN


In [42]:
# Check that our decodings match float16
def check_decodes(fmt, np_fmt, np_int, int_range):
    for i in range(int_range):
        val = decode_float(fmt, i)
        npval_us = np.array(val.fval)
        npval_np = np.array(i, dtype=np_int).view(dtype=np_fmt)
        assert np.array_equal(
            npval_us, npval_np, equal_nan=True
        ), f"{npval_us} != {npval_np}"

    print(f"No assertion failure - all {fmt.name} values decode consistently")


check_decodes(format_info_binary16, np.float16, np.uint16, 0xFFFF)

No assertion failure - all binary16 values decode consistently


In [43]:
# Check that our decodings match bfloat16
import ml_dtypes

check_decodes(format_info_bfloat16, ml_dtypes.bfloat16, np.uint16, 0xFFFF)

No assertion failure - all bfloat16 values decode consistently


In [44]:
# Check that our decodings match ml_dtypes 8-bit formats
import ml_dtypes

check_decodes(format_info_ocp_e4m3, ml_dtypes.float8_e4m3fn, np.uint8, 0xFF)

No assertion failure - all ocp_e4m3 values decode consistently
